# Chapter 6
## Decision Trees

# Training and Visualizing a Decision Tree

In [3]:
# load the libraries
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

In [4]:
# load the data
iris = load_iris(as_frame=True)
X=iris.data
y=iris.target

In [5]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int32

In [17]:
# train the data
X_iris = iris.data[["petal length (cm)", "petal width (cm)"]].values
y_iris = iris.target

tree_clf = DecisionTreeClassifier(max_depth=2, random_state=42)
tree_clf.fit(X_iris, y_iris)

DecisionTreeClassifier(max_depth=2, random_state=42)

In [13]:
from pathlib import Path

IMAGES_PATH = Path()/'images'/'decision_trees'
IMAGES_PATH.mkdir(parents=True,exist_ok=True)
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = IMAGES_PATH / f"{fig_id}.{fig_extension}"
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [18]:
# visualise the tree
from sklearn.tree import export_graphviz

export_graphviz(
        tree_clf,
        out_file=str(IMAGES_PATH / "iris_tree.dot"),  # path differs in the book
        feature_names=["petal length (cm)", "petal width (cm)"],
        class_names=iris.target_names,
        rounded=True,
        filled=True
    )